In [1]:
# importing module
from pymongo import MongoClient

In [2]:
from bson import json_util
import json as jsn

In [3]:
# Connect with the portnumber and host
client = MongoClient("mongodb://oobeya-mongo:27017/")

In [4]:
mydb = client["agilespaceDB"]
board_analysis = mydb["board_analyses"]

In [5]:
# board analyses in Oobeya
list(board_analysis.find({'addonType' : 'JIRA'}, {'_id':True, 'board.label':True}))

ServerSelectionTimeoutError: oobeya-mongo:27017: [Errno -3] Temporary failure in name resolution, Timeout: 30s, Topology Description: <TopologyDescription id: 63036f5210243888e4e7437d, topology_type: Unknown, servers: [<ServerDescription ('oobeya-mongo', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('oobeya-mongo:27017: [Errno -3] Temporary failure in name resolution')>]>

In [52]:
import datetime
import time

from_dt = datetime.datetime.strptime('2022-08-03','%Y-%m-%d')

In [53]:
def get_analysees():
    mydb = client["agilespaceDB"]
    issues = mydb["issues"]
    return list(issues.find(
        {
        'analysisId':'62e13eb8f264de0001707e7c',
        'issueChanges.status.updateDate': 
             {
            "$gte": from_dt,
            "$lte": datetime.datetime.now() 
             }
        },
        {
            'issueKey' : True,
            'issueCreatedDate' : True,
            'issueChanges.status.updateDate' : True,
            'issueChanges.status.fromString' : True,
            'issueChanges.status.toString' : True
        }

    ))

In [54]:
def diff(first_date, last_date):
    return last_date - first_date

In [55]:
def time_in_state_avg(data, count_info):
    for statu in data.keys():
        if count_info.get(statu) != 0:
            data.update([(statu, data.get(statu) / count_info.get(statu))])
    return data

In [56]:
def showResult():
    new_data = (time_in_state_avg(time_in_state, time_in_state_count_for_item))
    for stt in new_data:
        print(f"{stt} : {str(datetime.timedelta(seconds=new_data[stt]))}")
    

In [57]:
time_in_state = dict()
time_in_state_count_for_item = dict()
now = None

def initialize_time_in_state():
    return [{
        'To Do' : 0,
        'In Progress' : 0,
        'Code Review' : 0,
        'In Test' : 0
    },

    {
            'To Do' : 0,
            'In Progress' : 0,
            'Code Review' : 0,
            'In Test' : 0
    },

    datetime.datetime.now()]

In [60]:
# for cloud
time_in_state, time_in_state_count_for_item, now = initialize_time_in_state()
for k, item in enumerate(get_analysees()):
    c = 1
    status_list = item.get('issueChanges').get('status')
    if status_list != None : 
        for b, statu in enumerate(status_list):
            if statu.get('updateDate') > from_dt:
                c+=1
                #print(f"{item.get('issueKey')} {statu}")
        #print(c)
        #print(len(status_list))
        if len(status_list) < c:
            c = len(status_list)
        from_status_set = set()    
        for b in range(c):
            from_status = status_list[b].get('fromString')
            if from_status in time_in_state.keys():
                #print(f"c : {c} -- Total : {len(status_list)}")
                if from_status == 'To Do':
                    from_status_set.add(from_status)
                    diff_for_state = diff(item.get('issueCreatedDate'), status_list[b].get('updateDate'))
                    time_in_state.update([(from_status,time_in_state.get(from_status) + diff_for_state.total_seconds())])
                    print(f"For {item.get('issueKey')} --> {status_list[b].get('fromString')} to {status_list[b].get('toString')} - Time Elapsed in {status_list[b].get('fromString')} : {diff_for_state.total_seconds()} ")
                    print(f"{item.get('issueCreatedDate')} to {status_list[b].get('updateDate')}")
                if from_status != 'Done' and from_status != 'To Do':
                    from_status_set.add(from_status)
                    diff_for_state = diff(status_list[b+1].get('updateDate'), status_list[b].get('updateDate'))
                    print(f"For {item.get('issueKey')} --> {status_list[b].get('fromString')} to {status_list[b].get('toString')} - Time Elapsed in {status_list[b].get('fromString')} : {diff_for_state.total_seconds()} ")
                    print(f"{status_list[b+1].get('updateDate')} to {status_list[b].get('updateDate')}")
                    time_in_state.update([(from_status,time_in_state.get(from_status) + diff_for_state.total_seconds())])
                    from_status_set.add(from_status)
        
        #print(from_status_set)
        for x in from_status_set:
            time_in_state_count_for_item.update([(x, time_in_state_count_for_item.get(x) + 1)])
        

For DAS-2713 --> In Test to Done - Time Elapsed in In Test : 4273.219 
2022-08-09 12:05:36.981000 to 2022-08-09 13:16:50.200000
For DAS-2713 --> In Progress to In Test - Time Elapsed in In Progress : 1284.822 
2022-08-09 11:44:12.159000 to 2022-08-09 12:05:36.981000
For DAS-2713 --> To Do to In Progress - Time Elapsed in To Do : 993.141 
2022-08-09 11:27:39.018000 to 2022-08-09 11:44:12.159000
For DAS-2701 --> In Test to Done - Time Elapsed in In Test : 20089.798 
2022-08-08 07:28:12.380000 to 2022-08-08 13:03:02.178000
For DAS-2701 --> Code Review to In Test - Time Elapsed in Code Review : 5023.542 
2022-08-08 06:04:28.838000 to 2022-08-08 07:28:12.380000
For DAS-2701 --> In Progress to Code Review - Time Elapsed in In Progress : 260275.033 
2022-08-05 05:46:33.805000 to 2022-08-08 06:04:28.838000
For DAS-2701 --> To Do to In Progress - Time Elapsed in To Do : 167474.618 
2022-08-03 07:15:19.187000 to 2022-08-05 05:46:33.805000
For DAS-2698 --> In Test to Done - Time Elapsed in In Tes

In [61]:
showResult()

To Do : 45 days, 13:15:48.039261
In Progress : 2 days, 22:46:44.845100
Code Review : 23:32:26.090421
In Test : 1 day, 10:39:40.902650
